In [2]:
import pandas as pd

ndvi = pd.read_csv('ndvi_vals.csv')
bioclim = pd.read_csv('test2.csv')

In [7]:
bioclim = bioclim.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
ndvi = ndvi['NDVI']
bioclim['ndvi'] = ndvi
bioclim

,lat,lon,bio_13,bio_11,bio_9,bio_3,elev,bio_5,bio_6,bio_17,...,bio_15,bio_12,bio_1,bio_16,bio_8,bio_4,bio_19,bio_7,bio_14,bio_18
0,51.024975,13.708278,82.0,1.100000,2.116667,29.925650,199,24.6,-2.3,125.0,...,25.539827,663.0,9.683333,230.0,18.533333,710.95750,132.0,26.9,36.0,230.0
1,51.024975,13.708278,82.0,1.100000,2.116667,29.925650,199,24.6,-2.3,125.0,...,25.539827,663.0,9.683333,230.0,18.533333,710.95750,132.0,26.9,36.0,230.0
2,51.024975,13.791615,81.0,1.400000,2.483333,30.308643,117,24.9,-2.1,120.0,...,27.204247,645.0,10.008333,227.0,18.816666,709.33014,125.0,27.0,35.0,227.0
3,51.024975,13.791615,81.0,1.400000,2.483333,30.308643,117,24.9,-2.1,120.0,...,27.204247,645.0,10.008333,227.0,18.816666,709.33014,125.0,27.0,35.0,227.0
4,51.024975,13.791615,81.0,1.400000,2.483333,30.308643,117,24.9,-2.1,120.0,...,27.204247,645.0,10.008333,227.0,18.816666,709.33014,125.0,27.0,35.0,227.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33061,49.517217,8.201899,64.0,2.066667,3.200000,33.012820,141,24.8,-1.2,120.0,...,17.608332,593.0,10.075000,186.0,16.900000,664.93850,134.0,26.0,39.0,167.0
33062,49.463610,8.290625,68.0,2.183333,3.383333,33.460560,99,25.1,-1.1,121.0,...,19.536684,613.0,10.275000,198.0,17.100000,669.73030,133.0,26.2,39.0,177.0
33063,49.463143,8.290715,68.0,2.183333,3.383333,33.460560,99,25.1,-1.1,121.0,...,19.536684,613.0,10.275000,198.0,17.100000,669.73030,133.0,26.2,39.0,177.0
33064,49.463592,8.290598,68.0,2.183333,3.383333,33.460560,99,25.1,-1.1,121.0,...,19.536684,613.0,10.275000,198.0,17.100000,669.73030,133.0,26.2,39.0,177.0


In [11]:
bioclim.to_csv('bioclim.csv')


In [1]:
bioclim

NameError: name 'bioclim' is not defined